## Visualizing the filters and outputs of deep CNN layers for computer vision

In [33]:
import numpy as np
from scipy.misc import imresize
from skimage.color import gray2rgb
from mfeat import io
from mfeat import cnn as cnn_feat
from mfeat.bin import featuremap2
import json
from keras.models import Model, Sequential
from keras.layers import Conv2D, Activation
from keras import backend as K
from keras.applications.vgg16 import VGG16, preprocess_input
import bokeh
from bokeh.plotting import figure, show, output_notebook, output_file, reset_output
from bokeh.layouts import gridplot

cnn = VGG16(include_top=False, weights='imagenet')
layer_id = {layer.name: idx for idx, layer in enumerate(cnn.layers)}

ImportError: cannot import name 'inception_v3'

### First, some utility functions

* `deprocess_image`: turns a tensor of arbitrary numbers (usually weights) into an image array (uint8, [0-255])
* `add_alpha`: takes an MxNx3 image (RGB) and adds an alpha channel to make it MxNx4 (RGBA)
* `visualize_weights`: takes a cnn model instance and a layer number and generates an image plot to visualize filter weights

In [2]:
def deprocess_image(x,weights_array=False):
    
    # normalize tensor: center on 0., ensure std is 0.1
    
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if not weights_array:
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    
    return x

def img2rgba_vflip(im):
    """ traditional RGB image array to RGBA array with inverted y for backwards image_rgba method """
    # If grayscale, tile along dimension 2 first
    
    if im.shape[2]<3:
        im=np.tile(im,(1,1,3))

    im = im[::-1,:,:]
    imsize=im.shape
    
    # Tile a matrix of ones along dimension 2, convert to uint8 and range to 255
    return np.concatenate((im,np.ones([imsize[0],imsize[1],1]).astype('uint8')*255),axis=2)

def visualize_weights(cnn,layer,num_rows=8,num_cols=8):
    
    weights = cnn.layers[layer].get_weights()[0]  # 0 is the filter weights, 1 is the connection weights
    num_filters = weights.shape[3]
    
    # Deprocess the weights to make actual image arrays, add an alpha channel
    filters = [img2rgba_vflip((deprocess_image(weights[:,:,:,j],weights_array=True)))
               for j in range(num_filters)
              ]
    
    # Give them coordinates corresponding to the layer size
    xc,yc = [m.ravel().tolist() for m in np.meshgrid(range(num_cols),range(num_rows))]
    
    # Make Bokeh figure
    p=figure(active_scroll='wheel_zoom',x_range=[0,num_cols],y_range=[0,num_rows],width=400,height=400)
    p.image_rgba(filters,xc,yc,0.9,0.9)
    reset_output()
    output_notebook()
    show(p)
    
    return p
    
def visualize_outputs(output,num_rows=8,num_cols=8):
    
    num_filters = output.shape[3]
    
    # Deprocess the weights to make actual image arrays, add an alpha channel
    output_images = [image2rgba_vflip((deprocess_image(output[:,:,:,j])))
                     for j in range(num_filters)
                    ]
    
    # Give them coordinates corresponding to the layer size
    xc,yc = [m.ravel().tolist() for m in np.meshgrid(range(num_cols),range(num_rows))]
    
    # Make Bokeh figure
    p1=figure(active_scroll='wheel_zoom',x_range=[0,num_cols],y_range=[num_rows,0],width=400,height=400)
    p1.image_rgba(output_images,xc,yc,0.9,0.9)
    reset_output()
    output_notebook()
    
    return p1

### Using Bokeh's image_rgba, we can see all the filters

In [3]:
layer_num = 1
layer_name = cnn.layers[layer_num].name
visualize_weights(cnn,layer_num)

Loading BokehJS ...

Figure(id='17dd91ba-12f6-44b3-b7ba-fa93858bcfbf', ...)

### Here's a raw image to work with

In [18]:
image_file = "/Volumes/Macintosh HD/Users/nils/CC/afm-cnn/panther.jpg" #data/afm/33.tif"
image_gray = io.load_image(image_file)
if image_gray.ndim<3:
    image_rgb = gray2rgb(image_gray)
else:
    image_rgb = imresize(image_gray,(256,256))

p=figure(active_scroll='wheel_zoom',x_range=[0,1],y_range=[0,1],width=400,height=400)
p.image_rgba([img2rgba_vflip(image_rgb)],0,0,1,1)
reset_output()
output_file('raw_image.html')
show(p)

### Make a model that spits out the result of the first convolutional layer of VGG16

In [19]:
layer_num = 1
layer_name = cnn.layers[layer_num].name
model = Model(inputs=cnn.input, outputs=cnn.get_layer(layer_name).output)
out = model.predict(cnn_feat.image_tensor(image_rgb))

### Now produce a linked plot that shows filters, activations, and raw image alongside one another

In [20]:
weights = cnn.layers[layer_num].get_weights()[0]  # 0 is the filter weights, 1 is the connection weights
num_filters = weights.shape[3]

# Deprocess the weights to make actual image arrays, add an alpha channel
filters = [img2rgba_vflip((deprocess_image(weights[:,:,:,j],weights_array=True)))
           for j in range(num_filters)
          ]

# Deprocess the outputs
output_images = [img2rgba_vflip((deprocess_image(out[:,:,:,j])))
                 for j in range(num_filters)
                ]

# Give them coordinates corresponding to the layer size
num_rows=8
num_cols=8
xc = np.meshgrid(range(num_rows),range(num_cols))[0].ravel().tolist()
yc = np.meshgrid(range(num_rows),range(num_cols))[1].ravel().tolist()

# Make Bokeh figure
p_filt=figure(active_scroll='wheel_zoom',
              active_drag='pan',
              x_range=[0,num_rows],
              y_range=[0,num_cols],
              width=400,
              height=400,
              x_axis_label='Filters'
             )
p_filt.image_rgba(filters,xc,yc,0.9,0.9)

p_out=figure(active_scroll='wheel_zoom',
             active_drag='pan',
             x_range=p_filt.x_range,
             y_range=p_filt.y_range,
             width=400,
             height=400,
             x_axis_label='Filter Outputs'
            )
p_out.image_rgba(output_images,xc,yc,0.9,0.9)

p_im=figure(active_scroll='wheel_zoom',
            active_drag='pan',
            x_range=p_filt.x_range,
            y_range=p_filt.y_range,
            width=400,
            height=400,
            x_axis_label='Raw Images'
           )
p_im.image_rgba([img2rgba_vflip(image_rgb) for i in range(num_filters)],xc,yc,0.9,0.9)

p_grid = gridplot([[p_filt,p_im,p_out]],toolbar_location=None)
reset_output()
output_file('cnn_viz.html')
show(p_grid)

In [ ]:
weights[:,:,:,0]

### Now layer 2
The problem here is that this layer has 64 3x3 filters each with 64 "channels", one for each of the outputs from the previous layer's 64 filters. There are only 64 512x512 outputs, though, presumably one for each filter, summed over each channel's outputs.

In [ ]:
layer_num = 18
layer_name = cnn.layers[layer_num].name
model = Model(inputs=cnn.input, outputs=cnn.get_layer(layer_name).output)
out = model.predict(cnn_feat.image_tensor(image_gray))
print(layer_name)
print("Weight Shape")
try:
    print(cnn.layers[layer_num].get_weights()[0].shape)
except:
    print('MaxPool')
print("Output Shape")
print(out.shape)
print(cnn.layers[layer_num].get_config())

In [ ]:
cnn.layers[layer_num].get_config()

In [ ]:
out.shape

In [ ]:
p2=visualize_weights(cnn,2)

In [31]:
cnn_full.layers[-5:]

In [29]:
[i.get_config() for i in cnn_full.layers[-5:]]

[{'data_format': 'channels_last',
  'name': 'block5_pool',
  'padding': 'valid',
  'pool_size': (2, 2),
  'strides': (2, 2),
  'trainable': True},
 {'name': 'flatten', 'trainable': True},
 {'activation': 'relu',
  'activity_regularizer': None,
  'bias_constraint': None,
  'bias_initializer': {'class_name': 'Zeros', 'config': {}},
  'bias_regularizer': None,
  'kernel_constraint': None,
  'kernel_initializer': {'class_name': 'VarianceScaling',
   'config': {'distribution': 'uniform',
    'mode': 'fan_avg',
    'scale': 1.0,
    'seed': None}},
  'kernel_regularizer': None,
  'name': 'fc1',
  'trainable': True,
  'units': 4096,
  'use_bias': True},
 {'activation': 'relu',
  'activity_regularizer': None,
  'bias_constraint': None,
  'bias_initializer': {'class_name': 'Zeros', 'config': {}},
  'bias_regularizer': None,
  'kernel_constraint': None,
  'kernel_initializer': {'class_name': 'VarianceScaling',
   'config': {'distribution': 'uniform',
    'mode': 'fan_avg',
    'scale': 1.0,
    

In [30]:
layer_num = 18
layer_name = cnn_full.layers[layer_num].name
model_full = Model(inputs=cnn_full.input,
              outputs=cnn_full.get_layer(layer_name).output)
out = model_full.predict(cnn_feat.image_tensor(image_gray))
print(layer_name)
print("Weight Shape")
try:
    print(cnn_full.layers[layer_num].get_weights()[0].shape)
except:
    print('MaxPool')
print("Output Shape")
print(out.shape)
print(cnn_full.layers[layer_num].get_config())

ValueError: Error when checking : expected input_3 to have shape (None, 224, 224, 3) but got array with shape (1, 1024, 1024, 3)

NameError: name 'keras' is not defined

In [36]:
import keras
incept=keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)


87916544/87910968 [==============================] - 6s 0us/step


In [62]:
[l.name for l in incept.layers]

['input_5',
 'conv2d_95',
 'batch_normalization_95',
 'activation_95',
 'conv2d_96',
 'batch_normalization_96',
 'activation_96',
 'conv2d_97',
 'batch_normalization_97',
 'activation_97',
 'max_pooling2d_5',
 'conv2d_98',
 'batch_normalization_98',
 'activation_98',
 'conv2d_99',
 'batch_normalization_99',
 'activation_99',
 'max_pooling2d_6',
 'conv2d_103',
 'batch_normalization_103',
 'activation_103',
 'conv2d_101',
 'conv2d_104',
 'batch_normalization_101',
 'batch_normalization_104',
 'activation_101',
 'activation_104',
 'average_pooling2d_10',
 'conv2d_100',
 'conv2d_102',
 'conv2d_105',
 'conv2d_106',
 'batch_normalization_100',
 'batch_normalization_102',
 'batch_normalization_105',
 'batch_normalization_106',
 'activation_100',
 'activation_102',
 'activation_105',
 'activation_106',
 'mixed0',
 'conv2d_110',
 'batch_normalization_110',
 'activation_110',
 'conv2d_108',
 'conv2d_111',
 'batch_normalization_108',
 'batch_normalization_111',
 'activation_108',
 'activation_111

In [58]:
visualize_weights(incept,1,num_rows=4,num_cols=8)

Loading BokehJS ...

Figure(id='c3e67d17-a803-484a-862b-5825571e4013', ...)